In [ ]:
# This should be run with a 10_6_24 enviroment with higgs combine tool installed!

In [ ]:
from __future__ import print_function
import ROOT as r
import pickle
import re
import uncertainties
import numpy as np
import pandas as pd

In [ ]:
era = '2016'
if era=='2016':
    lumi=1.025
if era=='2017':
    lumi=1.023
if era=='2018':
    lumi=1.025

In [ ]:
outname="fits/limit_setting/abcd_dict_{}.pkl".format(era)
with open(outname,'rb') as f:
    abcd = pickle.load(f)

In [ ]:
outname="fits/limit_setting/fit_dict_split_{}.pkl".format(era)
with open(outname,'rb') as f:
    fit = pickle.load(f)

In [ ]:
def val2root(name, edges, values, std):
    nBins = len(edges) - 1
    th1f =  r.TH1F(name, name, nBins, edges)
    for i in range(th1f.GetNbinsX()):
        th1f.SetBinContent(i, values[i])
        th1f.SetBinError(i, std[i]**.5)
    return th1f

In [ ]:
x = r.RooRealVar('x', 'x', 110, 800)

In [ ]:
def make_dh(name, *args):
    fit_hist = val2root(name, *args)
    print(name, fit_hist, x)
    return r.RooDataHist(name, name, r.RooArgList(x), fit_hist)

In [ ]:
dh_signal_hists = []
norm_list = []
for (reg, mass, dbs), plot_dict in fit.items():
    hist_dict = plot_dict['hist']
    fit_dict = plot_dict['fit_hist']
    norm_list.append({"mass": mass, "hist": np.sum(hist_dict['nom']), "fit": np.sum(fit_dict['nom']), "reg": reg, "dbs":dbs})
    dbs = str(dbs).replace('.','p')
    name = '{}_BFFZprimeToMuMu_fit_M_{}_dbs{}'.format(reg, mass,dbs)
    hist_sum = np.sum(fit_dict['nom'])
    dh_signal_hists.append(make_dh(name, fit_dict['bins'], fit_dict['nom']/hist_sum,  fit_dict['std']/hist_sum*0))
    dh_signal_hists.append(make_dh(name+"_SysUp", fit_dict['bins'],
                                   (fit_dict['nom']+fit_dict['up'])/hist_sum,  fit_dict['std']/hist_sum*0))
    dh_signal_hists.append(make_dh(name+"_SysDown", fit_dict['bins'],
                                   (fit_dict['nom']+fit_dict['down'])/hist_sum,  fit_dict['std']/hist_sum*0))

    
    name = '{}_BFFZprimeToMuMu_hist_M_{}_dbs{}'.format(reg, mass,dbs)
    hist_sum = np.sum(fit_dict['nom'])
    dh_signal_hists.append(make_dh(name, hist_dict['bins'], hist_dict['nom']/hist_sum,  hist_dict['std']/hist_sum*0))
    dh_signal_hists.append(make_dh(name+"_SysUp", hist_dict['bins'],
                                   (hist_dict['nom']+hist_dict['up'])/hist_sum,  hist_dict['std']/hist_sum*0))
    dh_signal_hists.append(make_dh(name+"_SysDown", hist_dict['bins'],
                                   (hist_dict['nom']+hist_dict['down'])/hist_sum,  hist_dict['std']/hist_sum*0))


In [ ]:
print(pd.DataFrame(norm_list).round(2).to_latex())

In [ ]:
abcd_list = []
for reg, reg_dict in abcd.items():
    name = '{}_background'.format(reg)
    abcd_list.append(make_dh(name, reg_dict['bins'], reg_dict['nom'],  reg_dict['std']))

In [ ]:
abcd_list

In [ ]:
w = r.RooWorkspace("zp".format(era), "zp".format(era))

# Import abcd models
for dh in dh_signal_hists:
    print(dh)
    getattr(w, 'import')(dh)
    #break
    
for dh in abcd_list:
    print(dh)
    getattr(w, 'import')(dh)
    #break
    
    
w.Print()

# S a v e   w o r k s p a c e   i n   f i l e
# -------------------------------------------

# Save the workspace into a ROOT file
w.writeToFile("combine_data/era_{}_zp_fit_ws.root".format(era))


In [ ]:
combine_template = '''#higgs combine tool shape analysis card for z'to mumu 1 jet

-------------------------

imax 1  number of channels                                      #1 Jet
jmax 1  number of backgrounds -1                                    #following AN2015_207_v5, not sure why the -1 is there?
kmax *  number of nuisance parameters (sources of systematic uncertainties)

-------------------------

shapes data_obs * era_{era}_zp_fit_ws.root zp:SR{nJets}_background  #dummy data for now

shapes abcd SR1     era_{era}_zp_fit_ws.root zp:SR{nJets}_background          #parameterized ABCD pdf for modeling background

shapes BFFZp * era_{era}_zp_fit_ws.root zp:{signame} zp:{signame}_$SYSTEMATIC

-------------------------

bin       SR1
observation   -1

-------------------------

bin       SR1       SR1
process     abcd    BFFZp
process     1     -1
rate      -1   -1

-------------------------
lumi lnN -      {lumi}

#delatB1 lnN       1.03648698929    - 
#deltaS1 lnN       -       1.00771086929


Sys   shape  -           1'''

In [ ]:
for dh in dh_signal_hists:
    name = dh.GetName()
    print(name)
    nJets = re.findall(r'SR(\d)', name)[0]
    mass = re.findall(r'M_(\d+)_', name)[0]
    dbs = re.findall(r'dbs(\d)p(\d+)', name)[0]
    dbs = '{}.{}'.format(*dbs)
    sys = len(re.findall(r'dbs(\d)p(\d)+_', name)) > 0
    
    if sys: continue
    format_dict = {'era': era, 'mass': mass, 'nJets': nJets, 'dbs':dbs, 'lumi':lumi, 'signame':name}

    with open('combine_data/{}_{}.txt'.format(era,name), 'w') as f:
        print(f)
        f.write(combine_template.format(**format_dict))